In [1]:
#Data Analysis
import pandas as pd
import numpy as np

#Data Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set(font_scale=1)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

#Modeling
import fasttext
from sklearn.model_selection import cross_val_predict, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn_crfsuite import CRF, scorers, metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer
import scipy.stats
import eli5

In [2]:
path = "C:\\Users\\farih\\Downloads\\"
indoNLUmodel = "C:\\Users\\farih\\Downloads\\fasttext.4B.id.300.epoch5.uncased.bin"
model = fasttext.load_model(indoNLUmodel)

In [124]:
data = pd.read_csv("../Dataset/annotated_ner_data_new.csv")

In [125]:
data

,token,ner,tweet_id,index
0,Hidup,O,1382601382042103808,0
1,sesedih,O,1382601382042103808,1
2,dan,O,1382601382042103808,2
3,secaper,O,1382601382042103808,3
4,apa,O,1382601382042103808,4
...,...,...,...,...
170065,muntah,O,1386004972207153156,5
170066,Ngabuburit,O,1386301391086309382,0
170067,tadi,O,1386301391086309382,1
170068,sore,O,1386301391086309382,2


In [8]:
# A class to retrieve the sentences from the dataset
class getsentence(object):
    
    def __init__(self, data):
        self.n_sent = 1.0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["token"].values.tolist(),
                                                     s["ner"].values.tolist())]
        self.grouped = self.data.groupby("tweet_id").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [9]:
getter = getsentence(data)
sentences = getter.sentences
#This is how a sentence will look like. 
print(sentences[:1])

[[('@ridwankamil', 'O'), ('@KickAndyShow', 'O'), ('@Metro_TV', 'O'), ('Upaya', 'O'), ('untuk', 'O'), ('mnaikan', 'O'), ('nilai', 'O'), ('jual', 'O'), ('yg', 'O'), ('bgus,dngn', 'O'), ('cara', 'O'), ('d', 'O'), ('desain', 'O'), ('pak', 'O'), ('gubernur', 'B-PER'), ('akan', 'O'), ('menjadikn', 'O'), ('produk', 'O'), ('umkm', 'O'), ('d', 'O'), ('mintai', 'O'), ('krena', 'O'), ('yg', 'O'), ('desain', 'O'), ('org', 'O'), ('nomer', 'O'), ('satu', 'O'), ('di', 'O'), ('jabar', 'B-LOC')]]


In [10]:
words = list(set(data["token"].values))
n_words = len(words)
print(n_words)

35418


In [13]:
# Feature set
def word2features(sent, i):
    word = sent[i][0]
    # we=get_features(word) 
    # postag = sent[i][1]

    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower() if type(word) == str else word,
        'word[-3:]': word[-3:] if type(word) == str else word,
        'word[-2:]': word[-2:] if type(word) == str else word,
        'word[-1:]': word[-2:] if type(word) == str else word,
        'word[:3]': word[:3] if type(word) == str else word,
        'word[:2]': word[:2] if type(word) == str else word,
        'word[:1]': word[:2] if type(word) == str else word,
        'word.isupper()': word.isupper() if type(word) == str else word,
        'word.istitle()': word.istitle() if type(word) == str else word,
        'word.isdigit()': word.isdigit() if type(word) == str else word,
        # 'postag': postag,
        # 'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        # postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower() if type(word1) == str else word1,
            '-1:word.istitle()': word1.istitle() if type(word1) == str else word1,
            '-1:word.isupper()': word1.isupper() if type(word1) == str else word1,
            # '-1:postag': postag1,
            # '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        # postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower() if type(word1) == str else word1,
            '+1:word.istitle()': word1.istitle() if type(word1) == str else word1,
            '+1:word.isupper()': word1.isupper() if type(word1) == str else word1,
            # '+1:postag': postag1,
            # '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    #src: https://stackoverflow.com/questions/58736548/how-to-use-word-embedding-as-features-for-crf-sklearn-crfsuite-model-training
    we=model.get_word_vector(features['word.lower()'])
    for iv,value in enumerate(we):
        features['v{}'.format(iv)]=value

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

In [109]:
#Creating the train and test set
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

KeyboardInterrupt: 

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        train_size=0.8,
        test_size=0.2,
        # random but same for all run, also accurancy depends on the
        # selection of data e.g. if we put 10 then accuracy will be 1.0
        # in this example
        random_state=23,
        # keep same proportion of 'target' in test and target data
    )

In [20]:
y_train[2]

['O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'B-PER', 'O', 'O', 'O', 'O']

In [1]:
(X[0],y[0])

NameError: name 'X' is not defined

In [27]:
# group B and I results
# #First we select all the tags that are relevant for us i.e. remove the 'O' tag from the list. 

# labels = list(crf2.classes_)
labels = list(data['ner'].value_counts().keys())
labels = list(filter(lambda a: a != 'O', labels))
print(labels)
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

['B-PER', 'B-PROD', 'I-PER', 'B-LOC', 'B-ORG', 'I-PROD', 'I-ORG', 'I-EV', 'I-LOC', 'B-EV', 'I-WA', 'B-WA']


In [31]:
#Creating the CRF model
crf = CRF(algorithm='lbfgs',
          c1= 0.05912981852829489, 
          c2= 0.09113455878833218,
          max_iterations=100,
          all_possible_transitions=False)

In [32]:
%%time
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)
print(flat_classification_report(y_pred=pred, y_true=y, labels=sorted_labels, digits=3))   

d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['B-EV', 'I-EV', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD', 'B-WA', 'I-WA'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
              precision    recall  f1-score   support

        B-EV      0.604     0.262     0.366       809
     

In [18]:
#Tuning the parameters manually, setting c1 = 10
crf2 = CRF(algorithm='lbfgs',
          c1=1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [19]:
%%time
pred = cross_val_predict(estimator=crf2, X=X, y=y, cv=5)
print(flat_classification_report(y_pred=pred, y_true=y, labels=sorted_labels, digits=3))  

              precision    recall  f1-score   support

        B-EV      0.677     0.208     0.318       809
        I-EV      0.594     0.272     0.373       978
       B-LOC      0.769     0.633     0.694      2225
       I-LOC      0.585     0.471     0.522       957
       B-ORG      0.663     0.356     0.464      2085
       I-ORG      0.559     0.391     0.460      1124
       B-PER      0.899     0.634     0.744      5564
       I-PER      0.858     0.750     0.801      3058
      B-PROD      0.881     0.730     0.798      3554
      I-PROD      0.716     0.449     0.552      1177
        B-WA      0.286     0.012     0.023       164
        I-WA      0.190     0.015     0.027       271

   micro avg      0.801     0.566     0.663     21966
   macro avg      0.640     0.410     0.481     21966
weighted avg      0.779     0.566     0.648     21966

Wall time: 2min 39s


In [20]:
%%time
#Now we will create the Randomized CV search model wherein we will use a modified F1 scorer model considering only the relevant labels
# define fixed parameters and parameters to search
crf3 = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)   
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X, y)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 43.4min finished
Wall time: 44min 3s


RandomizedSearchCV(cv=3,
                   estimator=CRF(algorithm='lbfgs',
                                 all_possible_transitions=False, c1=0.1, c2=0.1,
                                 keep_tempfiles=None, max_iterations=100),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020B9F30DC48>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020B9F30D888>},
                   scoring=make_scorer(flat_f1_score, average=weighted, labels=['B-PER', 'B-PROD', 'I-PER', 'B-LOC', 'B-ORG', 'I-PROD', 'I-ORG', 'I-EV', 'I-LOC', 'B-EV', 'I-WA', 'B-WA']),
                   verbose=1)

In [21]:
#Lets check the best estimated parameters and CV score
print('Best parameters:', rs.best_params_)
print('Best CV score:', rs.best_score_)
print('Model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

Best parameters: {'c1': 0.05912981852829489, 'c2': 0.09113455878833218}
Best CV score: 0.6479905267319664
Model size: 3.65M


In [22]:
#Now we create the model again using the best estimators
crf3 = rs.best_estimator_
y_pred = crf3.predict(X)
print(metrics.flat_classification_report(y, y_pred, labels=sorted_labels, digits=3))

              precision    recall  f1-score   support

        B-EV      0.916     0.637     0.751       809
        I-EV      0.914     0.878     0.896       978
       B-LOC      0.954     0.967     0.961      2225
       I-LOC      0.942     0.964     0.953       957
       B-ORG      0.957     0.888     0.921      2085
       I-ORG      0.958     0.944     0.951      1124
       B-PER      0.980     0.967     0.974      5564
       I-PER      0.968     0.986     0.977      3058
      B-PROD      0.957     0.970     0.964      3554
      I-PROD      0.952     0.934     0.943      1177
        B-WA      0.949     0.902     0.925       164
        I-WA      0.911     0.908     0.909       271

   micro avg      0.960     0.942     0.951     21966
   macro avg      0.946     0.912     0.927     21966
weighted avg      0.959     0.942     0.950     21966



In [23]:
%%time
pred = cross_val_predict(estimator=crf3, X=X, y=y, cv=5)
print(flat_classification_report(y_pred=pred, y_true=y, labels=sorted_labels, digits=3))

              precision    recall  f1-score   support

        B-EV      0.608     0.250     0.354       809
        I-EV      0.580     0.324     0.416       978
       B-LOC      0.768     0.649     0.703      2225
       I-LOC      0.631     0.460     0.532       957
       B-ORG      0.646     0.398     0.493      2085
       I-ORG      0.564     0.382     0.455      1124
       B-PER      0.884     0.652     0.750      5564
       I-PER      0.868     0.751     0.806      3058
      B-PROD      0.880     0.750     0.810      3554
      I-PROD      0.721     0.455     0.558      1177
        B-WA      0.562     0.055     0.100       164
        I-WA      0.292     0.026     0.047       271

   micro avg      0.797     0.583     0.673     21966
   macro avg      0.667     0.429     0.502     21966
weighted avg      0.777     0.583     0.660     21966

Wall time: 2min 29s


In [24]:
crf3.fit(X,y)

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.05912981852829489,
    c2=0.09113455878833218, keep_tempfiles=None, max_iterations=100)

In [25]:
eli5.show_weights(crf3, top=30)

From \ To,O,B-EV,I-EV,B-LOC,I-LOC,B-ORG,I-ORG,B-PER,I-PER,B-PROD,I-PROD,B-WA,I-WA
O,2.474,0.339,-2.795,0.677,-2.823,1.335,-2.491,1.924,-3.902,1.097,-2.649,0.854,-2.339
B-EV,-1.061,1.556,5.686,-1.514,0.0,0.0,0.0,-0.484,0.0,0.302,0.0,0.0,0.0
I-EV,-0.562,0.251,5.676,-1.48,0.0,-1.479,0.0,-0.695,0.0,0.0,0.0,0.0,0.0
B-LOC,0.722,0.0,0.0,1.543,6.256,-0.071,0.0,-1.351,0.0,0.0,0.0,0.104,0.0
I-LOC,-0.249,0.0,0.0,0.329,6.094,0.0,0.0,-0.69,0.0,-0.176,0.0,0.0,0.0
B-ORG,0.445,0.639,0.0,-0.676,0.0,0.655,6.153,-0.531,0.0,-0.342,0.0,1.316,0.0
I-ORG,0.387,0.0,0.0,-1.941,0.0,-0.174,6.541,-0.67,0.0,0.268,0.0,0.0,0.0
B-PER,0.575,-0.758,0.0,-0.263,0.0,0.007,0.0,1.606,5.129,-0.338,0.0,0.0,0.0
I-PER,0.437,0.0,0.0,-0.464,0.0,-0.005,0.0,-0.174,4.28,0.0,0.0,0.0,0.0
B-PROD,0.564,-0.317,0.0,-0.963,0.0,-0.003,0.0,-0.328,0.0,1.774,5.33,0.0,0.0
